In [1]:
from theano.sandbox import cuda
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [3]:
print(len(idx))
for k,v in idx.items():
    print("k = %s, v = %s"%(k, v))
    break


88584
k = fawn, v = 34701


In [4]:
idx2word = {i:w for w,i in idx.items()}

In [5]:
idx2word[34701]

u'fawn'

In [6]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [7]:
print(' '.join([idx2word[i] for i in x_train[0]]))
print(len(x_train[1]))

bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't
433


In [8]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [9]:
vocal_size = 5000
trn1 = [np.array([j if j < vocal_size - 1 else vocal_size - 1 for j in i]) for i in  x_train]
test1 = [np.array([j if j < vocal_size - 1 else vocal_size - 1 for j in i]) for i in  x_test]
print(trn1[0])

[4999  309    6    3 1069  209    9 2175   30    1  169   55   14   46   82 4999   41  393  110  138
   14 4999   58 4477  150    8    1 4999 4999  482   69    5  261   12 4999 4999 2003    6   73 2436
    5  632   71    6 4999    1 4999    5 2004 4999    1 4999 1534   34   67   64  205  140   65 1232
 4999 4999    1 4999    4    1  223  901   29 3024   69    4    1 4999   10  694    2   65 1534   51
   10  216    1  387    8   60    3 1472 3724  802    5 3521  177    1  393   10 1238 4999   30  309
    3  353  344 2989  143  130    5 4999   28    4  126 4999 1472 2375    5 4999  309   10  532   12
  108 1470    4   58  556  101   12 4999  309    6  227 4187   48    3 2237   12    9  215]


In [10]:
lens = np.array(map(len, trn1))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

In [11]:
seq_len = 500
trn = [np.lib.pad(t, (0, seq_len - len(t)), 'constant', constant_values=(0)) if len(t) < seq_len else t[:seq_len] for t in trn1]
test = [np.lib.pad(t, (0, seq_len - len(t)), 'constant', constant_values=(0)) if len(t) < seq_len else t[:seq_len] for t in test1]
trn = np.array(trn)
test = np.array(test)

In [12]:
print(trn.shape)
print(test.shape)

(25000, 500)
(25000, 500)


In [13]:
lens = np.array(map(len, trn))
print(lens.max(), lens.min(), lens.mean())
a = [1, 2, 3, 4, 5]
np.lib.pad(a, (0,3), 'constant', constant_values=(0))

500 500 500.0


array([1, 2, 3, 4, 5, 0, 0, 0])

In [14]:
model = Sequential([
        Embedding(vocal_size, 32, input_length=seq_len),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.7),
        Dense(1, activation='sigmoid')
    ])

In [15]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [131]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 4s 159us/step - loss: 0.0979 - acc: 0.9674 - val_loss: 0.4342 - val_acc: 0.8471
Epoch 2/2
25000/25000 [==============================] - 4s 153us/step - loss: 0.0347 - acc: 0.9904 - val_loss: 0.4973 - val_acc: 0.8526


In [16]:
conv1 = Sequential([
        Embedding(vocal_size, 32, input_length=seq_len, dropout=0.2),
        Dropout(0.2),
        Convolution1D(64, 5, border_mode='same', activation='relu'),
        Dropout(0.2),
        MaxPooling1D(),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`


In [17]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 64)           10304     
_________________________________________________________________
dropout_3 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 64)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               1600100   
__________

In [18]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 16s 635us/step - loss: 0.4269 - acc: 0.7770 - val_loss: 0.2906 - val_acc: 0.8782
Epoch 2/2
25000/25000 [==============================] - 16s 630us/step - loss: 0.2275 - acc: 0.9100 - val_loss: 0.2731 - val_acc: 0.8868


In [135]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 16s 640us/step - loss: 0.1786 - acc: 0.9304 - val_loss: 0.2822 - val_acc: 0.8840
Epoch 2/5
25000/25000 [==============================] - 16s 634us/step - loss: 0.1453 - acc: 0.9467 - val_loss: 0.3112 - val_acc: 0.8754
Epoch 3/5
25000/25000 [==============================] - 16s 634us/step - loss: 0.1116 - acc: 0.9606 - val_loss: 0.3470 - val_acc: 0.8717
Epoch 4/5
25000/25000 [==============================] - 16s 634us/step - loss: 0.0866 - acc: 0.9698 - val_loss: 0.3896 - val_acc: 0.8718
Epoch 5/5
25000/25000 [==============================] - 16s 634us/step - loss: 0.0596 - acc: 0.9810 - val_loss: 0.4478 - val_acc: 0.8714


# Pre-Trained Vectors

In [23]:
!cd /home/ubuntu/

In [24]:
!pwd

/home/ubuntu/fastai_course/neilz/part1


In [26]:
def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('/home/ubuntu/data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))


In [27]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.50d'))

80117760/80107627 [==============================] - 3s 0us/step


In [37]:
print(vecs.shape)
print(len(words))
print(len(wordidx))
print(vecs[0])
print(words[0:10])
print(wordidx.keys()[:10])
print(wordidx.values()[:10])


50
400000
400000
[  4.1800e-01   2.4968e-01  -4.1242e-01   1.2170e-01   3.4527e-01  -4.4457e-02  -4.9688e-01
  -1.7862e-01  -6.6023e-04  -6.5660e-01   2.7843e-01  -1.4767e-01  -5.5677e-01   1.4658e-01
  -9.5095e-03   1.1658e-02   1.0204e-01  -1.2792e-01  -8.4430e-01  -1.2181e-01  -1.6801e-02
  -3.3279e-01  -1.5520e-01  -2.3131e-01  -1.9181e-01  -1.8823e+00  -7.6746e-01   9.9051e-02
  -4.2125e-01  -1.9526e-01   4.0071e+00  -1.8594e-01  -5.2287e-01  -3.1681e-01   5.9213e-04
   7.4449e-03   1.7778e-01  -1.5897e-01   1.2041e-02  -5.4223e-02  -2.9871e-01  -1.5749e-01
  -3.4758e-01  -4.5637e-02  -4.4251e-01   1.8785e-01   2.7849e-03  -1.8411e-01  -1.1514e-01
  -7.8581e-01]
['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]
['biennials', 'verplank', 'soestdijk', 'woode', 'mdbo', 'sowell', 'mdbu', 'woods', 'spiders', 'mdbs']
[130852, 42458, 274735, 311324, 212156, 65544, 119490, 2507, 19780, 285106]


In [59]:
def create_emb():
    facs = vecs.shape[1]
    emb = np.zeros((vocal_size, facs))
    for i in xrange(1, vocal_size):
        w = idx2word[i]
        if w in words:
            idx = wordidx[w]
            fac = vecs[idx]
            emb[i] = fac
        else:
            emb[i] = np.random.normal(loc = 0.0, scale=0.6, size= facs)
    emb[-1]  = np.random.normal(loc = 0.0, scale=0.6, size= facs)
    emb /= 3
    return emb

In [60]:
emb = create_emb()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [61]:
facs = vecs.shape[1]
print(vocal_size, facs)
np.zeros((vocal_size, facs))
print(min(idx2word.keys()))
print(emb.shape)

5000 50
1
(5000, 50)


In [62]:
conv2 = Sequential([
        Embedding(vocal_size, 50, input_length=seq_len, dropout=0.2, weights=[emb], trainable=False),
        Dropout(0.2),
        Convolution1D(64, 5, border_mode='same', activation='relu'),
        Dropout(0.2),
        MaxPooling1D(),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`


In [63]:
conv2.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 64)           16064     
_________________________________________________________________
dropout_5 (Dropout)          (None, 500, 64)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 250, 64)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               1600100   
__________

In [64]:
conv2.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 15s 607us/step - loss: 0.5774 - acc: 0.6902 - val_loss: 0.4733 - val_acc: 0.7783
Epoch 2/5
25000/25000 [==============================] - 15s 605us/step - loss: 0.4602 - acc: 0.7838 - val_loss: 0.4573 - val_acc: 0.7865
Epoch 3/5
25000/25000 [==============================] - 15s 608us/step - loss: 0.4137 - acc: 0.8117 - val_loss: 0.4098 - val_acc: 0.8176
Epoch 4/5
25000/25000 [==============================] - 15s 606us/step - loss: 0.3729 - acc: 0.8343 - val_loss: 0.4277 - val_acc: 0.8027
Epoch 5/5
25000/25000 [==============================] - 15s 605us/step - loss: 0.3364 - acc: 0.8508 - val_loss: 0.4306 - val_acc: 0.8030


In [67]:
conv2.layers[0].trainable=True
conv2.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [68]:
conv2.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 18s 703us/step - loss: 0.2412 - acc: 0.8984 - val_loss: 0.3375 - val_acc: 0.8594
Epoch 2/5
25000/25000 [==============================] - 18s 701us/step - loss: 0.1663 - acc: 0.9347 - val_loss: 0.3558 - val_acc: 0.8613
Epoch 3/5
25000/25000 [==============================] - 18s 701us/step - loss: 0.1206 - acc: 0.9542 - val_loss: 0.3691 - val_acc: 0.8623
Epoch 4/5
25000/25000 [==============================] - 18s 702us/step - loss: 0.0992 - acc: 0.9642 - val_loss: 0.3756 - val_acc: 0.8697
Epoch 5/5
25000/25000 [==============================] - 18s 703us/step - loss: 0.0767 - acc: 0.9709 - val_loss: 0.3929 - val_acc: 0.8687


In [70]:
conv2.optimizer.lr=1e-4

In [72]:
conv2.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 18s 711us/step - loss: 0.0297 - acc: 0.9893 - val_loss: 0.5429 - val_acc: 0.8734
Epoch 2/5
25000/25000 [==============================] - 18s 707us/step - loss: 0.0285 - acc: 0.9900 - val_loss: 0.5398 - val_acc: 0.8729
Epoch 3/5
25000/25000 [==============================] - 18s 705us/step - loss: 0.0258 - acc: 0.9907 - val_loss: 0.5303 - val_acc: 0.8728
Epoch 4/5
25000/25000 [==============================] - 18s 702us/step - loss: 0.0228 - acc: 0.9920 - val_loss: 0.6151 - val_acc: 0.8677
Epoch 5/5
25000/25000 [==============================] - 18s 702us/step - loss: 0.0228 - acc: 0.9924 - val_loss: 0.5828 - val_acc: 0.8726


# Multi-Size CNN

In [69]:
from keras.layers import Merge

In [75]:
graph_in = Input((vocal_size, 50))
convs = []
for s in range(3,6):
    x = Convolution1D(64, s, border_mode='same', activation='relu')(graph_in)
    y = MaxPooling1D()(x)
    conv = Flatten()(y)
    convs.append(conv)
    
out = Merge(mode='concat')(convs)
graph = Model(graph_in, out)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same", activation="relu")`
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 4, padding="same", activation="relu")`
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [84]:
conv3 = Sequential([
        Embedding(vocal_size, 50, input_length=seq_len, dropout=0.2, weights=[emb], trainable=False),
        Dropout(0.25),
        graph,
        Dropout(0.25),
        Dense(100, activation='relu'),
        Dropout(0.7),
        Dense(1, activation='sigmoid')
    ])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  from ipykernel import kernelapp as app


In [85]:
conv3.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
dropout_13 (Dropout)         (None, 500, 50)           0         
_________________________________________________________________
model_1 (Model)              multiple                  38592     
_________________________________________________________________
dropout_14 (Dropout)         (None, 48000)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               4800100   
_________________________________________________________________
dropout_15 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total para

In [86]:
conv3.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 36s 1ms/step - loss: 0.4636 - acc: 0.7845 - val_loss: 0.3983 - val_acc: 0.8234
Epoch 2/5
25000/25000 [==============================] - 36s 1ms/step - loss: 0.3955 - acc: 0.8275 - val_loss: 0.3902 - val_acc: 0.8260
Epoch 3/5
25000/25000 [==============================] - 36s 1ms/step - loss: 0.3595 - acc: 0.8440 - val_loss: 0.3846 - val_acc: 0.8256
Epoch 4/5
25000/25000 [==============================] - 36s 1ms/step - loss: 0.3301 - acc: 0.8566 - val_loss: 0.3785 - val_acc: 0.8306
Epoch 5/5
25000/25000 [==============================] - 36s 1ms/step - loss: 0.3002 - acc: 0.8718 - val_loss: 0.4026 - val_acc: 0.8183


# LTSM

In [81]:
conv4 = Sequential([
        Embedding(vocal_size, 50, input_length=seq_len, dropout=0.2, weights=[emb], trainable=False),
        Dropout(0.2),
        LSTM(100, consume_less="gpu"),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  from ipykernel import kernelapp as app
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, implementation=2)`


In [82]:
conv4.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
dropout_11 (Dropout)         (None, 500, 50)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 60,501
Non-trainable params: 250,000
_________________________________________________________________


In [83]:
conv4.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 157s 6ms/step - loss: 0.6927 - acc: 0.5053 - val_loss: 0.6931 - val_acc: 0.5058
Epoch 2/5
 1024/25000 [>.............................] - ETA: 1:52 - loss: 0.6925 - acc: 0.4961

KeyboardInterrupt: 